## Generate openSMILE Embeddings

### Install dependencies

In [1]:
!pip install opensmile -q

In [2]:
!pip install boxsdk -q
!pip install dotenv -q

In [3]:
!pip install numpy -q
!pip install torch -q

### Box Authorization

In [ ]:
from dotenv import load_dotenv
from boxsdk import OAuth2, Client
import os

load_dotenv()

auth = OAuth2(
    client_id=os.getenv('BOX_CLIENT_ID'),
    client_secret=os.getenv('BOX_CLIENT_SECRET'),
    access_token=os.getenv('BOX_ACCESS_TOKEN'),
)
client = Client(auth)

### Stream Data

In [ ]:
from pathlib import Path
from io import BytesIO
import soundfile as sf

def stream_data(file_path: Path | str, client: Client, folder_id: int):
    """
    file_path: relative path to a .wav file under the Box folder identified by folder_id
    client: authenticated boxsdk.Client
    folder_id: Box folder ID (root for the relative path)

    Uses the Box Search API scoped to the ancestor folder to find the file by name,
    downloads the file bytes and decodes the WAV into a waveform and sampling rate.

    Returns: (waveform (numpy array), sampling_rate)
    Raises FileNotFoundError if the file cannot be found.
    """
    # normalize parts and filename
    parts = [p for p in Path(file_path).parts if p not in ('.', '')]
    if not parts:
        raise FileNotFoundError(f'Empty path: {file_path}')
    filename = parts[-1]

    # use Box search scoped to the ancestor folder to find candidate files quickly
    # note: search is indexed server-side and is typically faster than walking folder listings
    results = client.search().query(query=filename, file_extensions=['wav'], ancestor_folder_ids=[str(folder_id)], limit=100)

    candidate = None
    for item in results:
        # ensure it's a file and the name matches exactly
        if getattr(item, 'type', None) == 'file' and getattr(item, 'name', None) == filename:
            # try to validate the returned item's ancestor path_collection if available
            try:
                # path_collection.entries is a list of ancestor dicts; skip the root entry (often 'All Files')
                ancestor_names = [e['name'] for e in item.path_collection['entries'][1:]] if getattr(item, 'path_collection', None) else []
            except Exception:
                ancestor_names = []

            # If ancestor_names is empty we can't validate here; accept the candidate (caller may validate manually)
            # Otherwise check that the tail of ancestor_names matches the path segments preceding the filename
            if not ancestor_names or ancestor_names[-len(parts[:-1]):] == parts[:-1] or len(parts) == 1:
                candidate = item
                break

    if not candidate:
        raise FileNotFoundError(f'File not found: {file_path} under folder {folder_id}')

    # download bytes (for large files prefer download_to with a stream to disk)
    file_bytes = client.file(candidate.id).content()
    bio = BytesIO(file_bytes)

    # decode waveform using soundfile (reads from file-like objects)
    waveform, sr = sf.read(bio)
    return waveform, sr

### openSMILE Feature Extractor

In [ ]:
import opensmile

smile = opensmile.Smile(
    feature_set=opensmile.FeatureSet.eGeMAPSv02,  # ComParE_2016 or emobase or eGeMAPSv02
    feature_level=opensmile.FeatureLevel.Functionals,
)

def extract_features(waveform, sampling_rate):
    """
    waveform: waveform of a .wav file
    sampling_rate: sampling rate of a .wav file

    Returns extracted features using the openSMILE feature extractor

    """
    return smile.process_signal(waveform, sampling_rate)

### Save Embeddings

In [ ]:
def save_embedding(features, out_path: Path):
    if os.path.exists(out_path):
        print(f"✅ Already exists: {out_path}, skipping.")
        return
    np.save(features, out_path)

### Extract and Save Embeddings

In [ ]:
def extract_and_save_embeddings(csv_path: Path, out_path: Path, client: Client, folder_id: int, session_nums: list[int]) -> None:
    pass

#### Paths

In [ ]:
DATA_DIR = Path('../../data/iemocap_4way_data')
TRAIN_PATH = DATA_DIR / 'train_4way_with_minus_one.csv'
VAL_PATH = DATA_DIR / 'val_4way_with_minus_one.csv'
TEST_PATH = DATA_DIR / 'test_4way_with_minus_one.csv'

SAVE_DIR = Path('../../data/embeddings/4way_opensmile/eGeMAPSv02')
SAVE_TRAIN_PATH = SAVE_DIR / 'train.npz'
SAVE_VAL_PATH = SAVE_DIR / 'val.npz'
SAVE_TEST_PATH = SAVE_DIR / 'test.npz'

FOLDER_ID = 279758036893

#### Extract and Save Training Embeddings

In [ ]:
extract_and_save_embeddings(TRAIN_PATH, SAVE_TRAIN_PATH, client, FOLDER_ID, [1, 2, 3])

#### Extract and Save Validation Embeddings

In [ ]:
extract_and_save_embeddings(TRAIN_PATH, SAVE_TRAIN_PATH, client, FOLDER_ID, [4])

#### Extract and Save Testing Embeddings

In [ ]:
extract_and_save_embeddings(TRAIN_PATH, SAVE_TRAIN_PATH, client, FOLDER_ID, [5])